In [1]:
import pandas as pd
import numpy as np
file_path = './data/cleanDataset.csv'

df = pd.read_csv(file_path)



In [2]:
df.head(10)

,Unnamed: 0,age,weight,height,bmi,bmr,Cal/Day,is_female,is_male
0,0,33,72.988352,163.239187,27.390800,1478.095,1424.128437,True,False
1,1,31,73.126322,163.251432,27.438460,1488.840,1435.584663,True,False
2,2,29,68.103992,163.304792,25.537284,1450.116,1395.694872,True,False
3,3,31,74.244270,163.135282,27.897620,1688.023,1612.038214,False,True
4,4,23,61.811747,163.177729,23.213961,1572.310,1527.978279,False,True
5,5,20,57.532279,163.220488,21.595449,1390.785,1334.450839,True,False
6,6,30,73.164338,163.087471,27.507952,1679.789,1605.940067,False,True
7,7,39,86.369421,163.272569,32.399146,1578.410,1528.147772,True,False
8,8,45,93.709650,163.121760,35.217662,1859.431,1736.607495,False,True
9,9,16,53.014752,159.672736,20.793864,1481.866,1453.102119,False,True


In [3]:
df.drop(columns=["bmr"])

,Unnamed: 0,age,weight,height,bmi,Cal/Day,is_female,is_male
0,0,33,72.988352,163.239187,27.390800,1424.128437,True,False
1,1,31,73.126322,163.251432,27.438460,1435.584663,True,False
2,2,29,68.103992,163.304792,25.537284,1395.694872,True,False
3,3,31,74.244270,163.135282,27.897620,1612.038214,False,True
4,4,23,61.811747,163.177729,23.213961,1527.978279,False,True
...,...,...,...,...,...,...,...,...
8210,8210,27,66.838372,163.154531,25.108894,1392.099537,True,False
8211,8211,23,60.658746,163.315070,22.742641,1351.306644,True,False
8212,8212,25,63.111686,163.318335,23.661370,1365.856458,True,False
8213,8213,20,56.724673,163.104114,21.322699,1491.647444,False,True


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


y = df['Cal/Day']
X = df.drop(columns=["Cal/Day", "Unnamed: 0", "bmr"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [5]:
X_train

,age,weight,height,bmi,is_female,is_male
434,15,54.386855,160.762119,21.043913,False,True
5735,31,77.307339,163.147714,29.044155,True,False
4086,42,89.775712,163.264113,33.680412,True,False
7255,88,98.164001,163.121129,36.891967,False,True
1489,38,86.315833,163.082328,32.454630,False,True
...,...,...,...,...,...,...
5734,20,57.445012,163.096164,21.595578,False,True
5191,85,96.583596,163.158533,36.281379,True,False
5390,29,68.003117,163.081671,25.569281,False,True
860,15,49.045121,159.553881,19.265535,False,True


In [6]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score

GBR = GradientBoostingRegressor()
LR = LinearRegression()


GBR.fit(X_train, y_train.values)
LR.fit(X_train, y_train.values)



LinearRegression()

Results for GBR:

In [7]:
y_pred = GBR.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error = {mse}')
print('R2 Score',r2_score(y_test, y_pred))

Mean Squared Error = 29.963649942161286
R2 Score 0.9983013541279319


Results for Linear Regression:

In [8]:
y_pred = LR.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error = {mse}')
print('R2 Score',r2_score(y_test, y_pred))

Mean Squared Error = 1.035942396408135e-25
R2 Score 1.0


In [9]:
import os
import joblib

# Create a directory if it doesn't exist
models_directory = './model/'
os.makedirs(models_directory, exist_ok=True)


# Save the trained model and scaler to files
model_filename = os.path.join(models_directory, 'calorie_prediction_model.joblib')

joblib.dump(LR, model_filename)

['./model/calorie_prediction_model.joblib']

In [10]:
def get_prediction():
    age = int(input("Enter your age: "))
    weight = float(input("Enter your weight in kgs: "))
    height = float(input("Enter your Height in M: "))
    bmi = weight/(height * height)
    gender = input("Enter you gender: M/F")
    '''age = 24
    weight = 68.0
    height = 1.78
    bmi = weight/(height * height)
    gender = "m" '''

    model=joblib.load('./model/calorie_prediction_model.joblib')

    if gender.lower() == "male" or gender.lower() == "m":
        is_male = 1
        is_female= 0
    else:
        is_female = 1
        is_male = 0

    print("GRB says: %f Cal/D"%(GBR.predict(np.array([age, weight, height*100, bmi, is_female, is_male]).reshape(1, -1))))
    print("LR says: %f Cal/D"%(model.predict(np.array([age, weight, height*100, bmi, is_female, is_male]).reshape(1, -1))))